# Interactive Graphics Sandbox

Can we use the `bokeh` library to make an interactive image plot? All we need is to be able to capture an (x,y) pixel position on an image, so we can use it in our lens model. Let's start by following [this tutorial](https://pyscience.wordpress.com/2014/09/01/interactive-plotting-in-ipython-notebook-part-12-bokeh/) and see how far we get.

In [3]:
# !conda install bokeh

In [4]:
import numpy
import bokeh.plotting

ImportError: No module named bokeh.plotting

In [5]:
# Set the output to 'notebook' mode
bokeh.plotting.output_notebook()

NameError: name 'bokeh' is not defined

In [6]:
# Simple numpy calculation of a sine wave
x = numpy.arange(0.0, 100.0, 0.1)
y = numpy.sin(x)

In [7]:
p = bokeh.plotting.line(x,y, line_width=3.0)
bokeh.plotting.show()

NameError: name 'bokeh' is not defined